In [13]:
#https://www.datacamp.com/tutorial/xgboost-in-python
import seaborn as sns

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import warnings


warnings.filterwarnings("ignore")


In [14]:
import ssl

#bypass the ssl error
ssl._create_default_https_context = ssl._create_unverified_context

diamonds = sns.load_dataset("diamonds")

diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [15]:
# check the shape of the df
diamonds.shape

(53940, 10)

In [16]:
diamonds.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [17]:
diamonds.describe(exclude=np.number)

,cut,color,clarity
count,53940,53940,53940
unique,5,7,8
top,Ideal,G,SI1
freq,21551,11292,13065


In [18]:
# check for nulls
null_counts = diamonds.isnull().sum()
print(null_counts)

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64


In [19]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]
print(X)
print(y)

       carat        cut color clarity  depth  table     x     y     z
0       0.23      Ideal     E     SI2   61.5   55.0  3.95  3.98  2.43
1       0.21    Premium     E     SI1   59.8   61.0  3.89  3.84  2.31
2       0.23       Good     E     VS1   56.9   65.0  4.05  4.07  2.31
3       0.29    Premium     I     VS2   62.4   58.0  4.20  4.23  2.63
4       0.31       Good     J     SI2   63.3   58.0  4.34  4.35  2.75
...      ...        ...   ...     ...    ...    ...   ...   ...   ...
53935   0.72      Ideal     D     SI1   60.8   57.0  5.75  5.76  3.50
53936   0.72       Good     D     SI1   63.1   55.0  5.69  5.75  3.61
53937   0.70  Very Good     D     SI1   62.8   60.0  5.66  5.68  3.56
53938   0.86    Premium     H     SI2   61.0   58.0  6.15  6.12  3.74
53939   0.75      Ideal     D     SI2   62.2   55.0  5.83  5.87  3.64

[53940 rows x 9 columns]
       price
0        326
1        326
2        327
3        334
4        335
...      ...
53935   2757
53936   2757
53937   2757
5393

In [20]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()
print(cats)

# # Convert to Pandas category
# for col in cats:
#    X[col] = X[col].astype('category')

# print(X)

['cut', 'color', 'clarity']


In [21]:
X.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
x           float64
y           float64
z           float64
dtype: object

In [22]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [23]:
print(len(X_train))
print(len(X_test))

43152
10788


In [24]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [28]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

In [ ]:
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=100,
)

In [ ]:
model

In [ ]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)

In [ ]:
print(preds)

[ 503.4773 5810.302  2587.0713 ... 2036.3007 6246.833  1369.7161]


In [ ]:
rmse = mean_squared_error(y_test, preds, squared=False)

print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 529.291


EVALUATION SETS

In [ ]:
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

In [ ]:
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=100,
   evals=evals
)

[0]	train-rmse:2869.39664	validation-rmse:2824.71766
[1]	train-rmse:2088.76624	validation-rmse:2058.64417
[2]	train-rmse:1548.93014	validation-rmse:1528.25913
[3]	train-rmse:1184.09574	validation-rmse:1175.69096
[4]	train-rmse:941.26700	validation-rmse:939.34287
[5]	train-rmse:786.16941	validation-rmse:795.06946
[6]	train-rmse:689.33568	validation-rmse:702.86336
[7]	train-rmse:626.32308	validation-rmse:644.64658
[8]	train-rmse:586.56146	validation-rmse:608.05544
[9]	train-rmse:563.28094	validation-rmse:589.66871
[10]	train-rmse:547.07965	validation-rmse:577.17280
[11]	train-rmse:535.11872	validation-rmse:568.05455
[12]	train-rmse:525.26585	validation-rmse:561.00563
[13]	train-rmse:518.80786	validation-rmse:556.84295
[14]	train-rmse:513.46191	validation-rmse:554.75324
[15]	train-rmse:508.12670	validation-rmse:551.29301
[16]	train-rmse:504.46784	validation-rmse:547.86839
[17]	train-rmse:497.82264	validation-rmse:544.97547
[18]	train-rmse:495.93198	validation-rmse:544.37612
[19]	train-rms

In [ ]:
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=300,
   evals=evals,
   verbose_eval=10, # Every ten rounds
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2869.39664	validation-rmse:2824.71766
[10]	train-rmse:547.07965	validation-rmse:577.17280
[20]	train-rmse:489.24714	validation-rmse:541.19490
[30]	train-rmse:463.58497	validation-rmse:534.88041
[40]	train-rmse:447.28130	validation-rmse:533.14998
[50]	train-rmse:433.37406	validation-rmse:531.71893
[60]	train-rmse:417.71834	validation-rmse:530.52044
[70]	train-rmse:406.99344	validation-rmse:529.95334
[80]	train-rmse:396.80919	validation-rmse:529.73025
[90]	train-rmse:386.60973	validation-rmse:528.87842
[100]	train-rmse:376.03018	validation-rmse:529.35955
[110]	train-rmse:366.61975	validation-rmse:529.96552
[120]	train-rmse:356.60058	validation-rmse:529.77331
[130]	train-rmse:349.71051	validation-rmse:528.67954
[140]	train-rmse:341.89822	validation-rmse:528.79583
[150]	train-rmse:335.45942	validation-rmse:528.55706
[160]	train-rmse:330.25106	validation-rmse:528.81294
[170]	train-rmse:322.29198	validation-rmse:528.59957
[180]	train-rmse:315.68150	validation-rmse:529.56318
[1